<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 657: Microwave Remote Sensing<b> </font>

<font size="5"> <b>Lab 5: InSAR Processing using ESA's SNAP Toolbox <font color='rgba(200,0,0,0.2)'> -- [20 Points] </font> </b> </font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks</b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /><font color='rgba(200,0,0,0.2)'> <b>Due Date: </b> April 05, 2019 </font>
</font>

<font size="3"> This Lab is part of the UAF course <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657: Microwave Remote Sensing</a>. It is introducing you to the concepts of interferometric SAR (InSAR) processing using the European Space Agency's (ESA) Sentinel Application Platform (SNAP), a free software tool specifically designed to analyze data from all of ESA's Sentinel Sensors. In this lab, we specifically will be working with the Sentinel-1 constellation, a two-sensor C-band radar providing 10m-resolution SAR data globally every 6-12 days. As previously, all data analysis will be done in the framework of *Jupyter Notebooks*.  <br>

<b>In this chapter we introduce the following data analysis concepts:</b>

- What is special about the Sentinel-1 SAR acquisition mode
- How to read visualize and work with Sentinel-1 data within the SNAP toolbox.
- How to process InSAR data start-to-finish within SNAP.
</font>

<font size="4"> <font color='rgba(200,0,0,0.2)'> <b>THIS NOTEBOOK INCLUDES <u>THREE</u> HOMEWORK ASSIGNMENTS at its very end.</b></font> Complete all assignments to achieve full score. </font> <br>
<font size="3"> To submit your homework, please download your Jupyter Notebook from the server both asf PDF (*.pdf) and Notebook file (*.ipynb) and submit them as a ZIP bundle via Blackboard or email (to fjmeyer@alaska.edu). To download, please select the following options in the main menu of the notebook interface:

<ol type="1">
  <li><font color='rgba(200,0,0,0.2)'> <b> Save your notebook with all of its content</b></font> by selecting <i> File / Save and Checkpoint </i> </li>
  <li><font color='rgba(200,0,0,0.2)'> <b>To export in Notebook format</b></font>, click on <i>File / Download as / Notebook (.ipynb)</i>  <font color='gray'>--- Downloading your file may take a bit as the Notebook will be about 100MB in size</font></li>
  <li><font color='rgba(200,0,0,0.2)'> <b>To export in PDF format</b></font>, click on <i>File / Download as / PDF vs LaTeX (.pdf) </i></li>
</ol>

Contact me at fjmeyer@alaska.edu should you run into any problems.
</font>

</font>

<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>


<font size="3">In this notebook we will use the following scientific library:
<ol type="1">
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
</font>   
    <font face="Calibri" size="3">The first step is to <b>import all required python modules:</b></font>


In [1]:
import gdal # for translate
#import numpy as np
#import matplotlib.pylab as plt

import requests # for get
from getpass import getpass

# ASF's Jupyter Notebook Module
from asf_notebook import path_exists
from asf_notebook import download_ASF_granule

#import sys
import os # for chdir, getcwd, path.basename, path.exists, path.isfile, stat, system
import re # for split
#import datetime
import time # for perf_counter
import glob #for glob
#import csv

<font face="Calibri" size="3"><b>Set up matplotlib plotting</b> inside the notebook:</font>

In [2]:
%matplotlib inline

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Background on Analyzed Event </b> </font> <img src="NotebookAddons/Shakemap.jpg" width="350" align="right" /> 

<font size="3"> Interferometric SAR processing exploits the difference between the phase signals of repeated SAR acquisitions to analyze the shape and deformation of the Earth surface. The principles and concepts of Interferometric SAR (InSAR) processing were covered in Lectures 12 to 14 of <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657</a>. Please refer to information in the <a href="https://radar.community.uaf.edu/module-iii-advanced-radar-concepts/" target="_blank">lecture notes</a> should you have InSAR-related questions that are not explained in this document.

In the first part of this lab, we will analyze a pair of Sentinel-1 images bracketing the devastating 2016 Kumamoto earthquake, whose 6.5 magnitude foreshock and 7.0 main shock devastated large areas around Kumamoto, JP on April 14th and 16th, respectively. The figure to your right shows the USGS ShakeMap associated with the 7.0 magnitude main shock, showing both the violence of the event and the location of the largest devastation.

We will use ESA’s Sentinel Application Platform (SNAP) to perform InSAR processing on these Sentinel-1 images. The advantages of the SNAP tool include (1) its graphical user interface, which renders the SNAP tool straightforward to use (compared to other InSAR processing tools); (2) the easy-to-access, free-of-charge, and public domain nature of the SNAP tool; and (3) the fact that SNAP is an integrative multi-sensor toolbox and enables processing data from all Sentinel sensors within one joint processing platform. While the graphical user interface is not used here, it is available to you on your personal computer for future exercises.
</font>

<hr>
<font face="Calibri">

<font size="5"> <b> 2. Sentinel-1 Data used in this Lab </b> </font> <img src="NotebookAddons/Lab5-topsar.jpg" width="300" align="right" /> 

<font size="3"> The Interferometric Wide (IW) swath mode is the main acquisition mode over land for <a href="https://www.asf.alaska.edu/sentinel/" target="_blank">Sentinel-1</a>. It acquires data with a 250 km swath at 5 m by 20 m spatial resolution (single look). IW captures three sub-swaths using the Terrain Observation with Progressive Scans SAR (TOPSAR) acquisition principle. With the TOPSAR technique, in addition to steering the beam in range as in ScanSAR, the beam is also electronically steered from backward to forward in the azimuth direction for each burst, avoiding scalloping and resulting in homogeneous image quality throughout the swath. A schematic of the TOPSAR acquisition principle is shown to your right. For more information on TOPSAR please read <cite><a href="https://ieeexplore.ieee.org/abstract/document/1677745"><i>TOPSAR: Terrain Observation by Progressive Scans</i></a> by De Zan & Monti Guarnieri</cite>. 

IW Single Look Complex (SLC) products contain one image per sub-swath and one per polarization channel, for a total of three (single polarization) or six (dual polarization) images in an IW product. Each sub-swath image consists of a series of bursts, where each burst has been processed as a separate SLC image. The individually focused complex burst images are included, in azimuth-time order, into a single sub-swath image with black-fill demarcation in between.
<br><br>

<img src="NotebookAddons/Lab5-footprint.jpg" width="400" align="right" /> We will use a pair of repeated Sentine-1 images for this part of the lab that were acquired on April 8 and April 20, 2016, bracketing the fore- and main shock of the Kumamoto earthquake event. Hence, the phase difference between these image acquisitions capture the cumulative co-seismic deformation caused by both of these seismic events. The footprint of the Sentinel-1 images is shown to your right and covers the areas affected by the earthquake. Hence, Sentinel-1 data are a good basis for studying earthquake-related surface deformation.

We will download the <b>SLC process level</b> for the following images directly from the <b><a href="https://vertex.daac.asf.alaska.edu/#" target="_blank">ASF VERTEX, Find Data API</a></b>:
- <b>Pre-event image:</b> S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB
- <b>Post-event image:</b> S1A_IW_SLC__1SSV_20160420T091355_20160420T091423_010903_010569_F9CE


</font> 
</font>

<font face="Calibri" size="3"><b>Authenticate yourself on ASF Vertex with your Earthdata login</b> so you can download the files needed for this exercise. To do so, replace the <i>NASA_earthdata_username</i> "fmeyer" with your own username in the code cell below and run the cell: </font>

In [3]:
NASA_earthdata_username = 'aflewandowski'
NASA_earthdata_password = getpass()

with open("/home/jovyan/.netrc", "w") as f:
    f.write(f"machine urs.earthdata.nasa.gov login {NASA_earthdata_username} password {NASA_earthdata_password}\n")

········


<font face="Calibri" size="3"> Next, <b>create a working directory for this analysis and change into it.</b></font>

In [4]:
path = "/home/jovyan/notebooks/ASF/GEOS_657_Labs"
if path_exists(path):
    os.chdir(path)
    !mkdir -p lab_5_data/Kumamoto
    path = f"{path}/lab_5_data/Kumamoto"
    if path_exists(path):
        os.chdir(path)
print(f"current directory: {os.getcwd()}")

current directory: /home/jovyan/notebooks/ASF/GEOS_657_Labs/lab_5_data/Kumamoto


<font face="Calibri" size="3"> Now you are ready to <b>download the files</b> by passing the granule names and process levels to download_ASF_granule(). </font>

In [5]:
pre_event = download_ASF_granule("S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB", "SLC")
post_event = download_ASF_granule("S1A_IW_SLC__1SSV_20160420T091355_20160420T091423_010903_010569_F9CE", "SLC")

[==================================================] 47747229.0 bps, 100%    
Done

[==================================================] 48756907.0 bps, 100%    
Done



<br>
<hr>
<font face="Calibri" size="5"> <b> 3. InSAR Processing using SNAP: </b> 

<font size="3"> We now will progress through the individual steps of the InSAR Processing flow using the <a href="http://step.esa.int/main/download/" target="_blank">SNAP toolbox</a>. We must first <b>set up the path to the SNAP toolbox and define the names of the image files we want to process:</b></font>

</font> 

In [ ]:
base_snap = "/usr/local/snap/bin/gpt "

In [ ]:
pre_event_base_granule = re.split('/', pre_event.replace('.zip', ''))[-1]
post_event_base_granule = re.split('/', post_event.replace('.zip', ''))[-1]

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.1 Extract Sub-swath from the Data Files </b> </font>

<font face="Calibri" size="3"> SNAP natively processes the individual sub-swaths of a Sentinel-1 data set individually. Hence, we have to first extract the targeted sub-swath data from the data file. We will start with IW1, mostly because it is centered nicely on the event.</font> 

<font face="Calibri" size="3"> <b>Write a function using TOPSAR-Split to create the sub-swaths:</b></font> 

In [ ]:
def topsar_split(in_data, ss, td2, subs):
    call_flag = f"TOPSAR-Split -Psubswath={ss} -PselectedPolarisations=VV "
    if subs != "":
        call_flag = f"{call_flag}-PwktAoi=\"subs\" "
    out = f"-t {td2}/{in_data.replace('.zip', '_TS')} "
    in_d = f"-Ssource={in_data}"
    cmd = f"{base_snap}{call_flag}{out}{in_d}"
    print(f"Spliting subswaths in {in_data}")
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print("\nDone.")
    return f"{in_data.replace('.zip', '_TS.dim')}"

<font face="Calibri" size="3"> Use our topsar_split() function to <b>split the pre-event data:</b></font> 

In [ ]:
pre_event_split = topsar_split(pre_event, 'IW1', os.getcwd(), '')

<font face="Calibri" size="3"> Then <b>split the post-event data.</b></font> 

In [ ]:
post_event_split = topsar_split(post_event, 'IW1', os.getcwd(), '')

<font face="Calibri" size="3">Each .zip file has been split into a .data directory and a .dim file. Run the code cell below to <b>view the files in your current working directory for varification.</b> <i>Note that "_TS" has been appended to each filename to signify that they were processed with TOPSAR-Split</i>:</font> 

In [ ]:
#optional. Uncomment to run.
#!ls --color

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.2 Apply Orbit File </b> </font>

<font face="Calibri" size="3"> We are interested in measuring surface deformation for this analysis. Remember that the interferometric phase $\phi$ contains (at a minimum) the following information:

$\phi = \phi_{topo} + \phi_{defo} = \frac{4\cdot\pi}{\lambda} \frac{B_{\perp}}{R\cdot cos(\theta)} \cdot h + \frac{4\cdot\pi}{\lambda} \cdot \Delta R $
<br><br>
In order to be able to remove most of $\phi_{topo}$ from this equation, we need a good DEM as well as accurate information about the interferometric baseline $B_{\perp}$. 
<br><br>
<b>Write a function to update the orbit information in the data file to the post-processed <i>precise</i> orbits</b>.
</font> 


In [ ]:
def apply_orbit(in_data, td2):
    apply_orbit_flag = "Apply-Orbit-File "
    orbit_type = f"-PcontinueOnFail=\"true\" -PorbitType=\"Sentinel Precise (Auto Download)\" "
    out = f"-t {td2}/{in_data.replace('.dim', '_OB')} " 
    cmd = f"{base_snap}{apply_orbit_flag}{out}{orbit_type} -Ssource={td2}/{in_data}"
    print(f"Applying Precise Orbit Information to {in_data}...")
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print("\nDone.")
    return f"{in_data.replace('.dim', '_OB.dim')}"

<font face="Calibri" size="3"> Use our apply_orbit() function to <b>update the orbit information on the pre-event .dim:</b></font> 

In [ ]:
pre_event_orbit_output = apply_orbit(pre_event_split, os.getcwd())
print(f"\npre_event_orbit_output: {pre_event_orbit_output}\n")

<font face="Calibri" size="3"><b>Update the orbit information on the post-event .dim:</b></font> 

In [ ]:
post_event_orbit_output = apply_orbit(post_event_split, os.getcwd())
print(f"\npost_event_orbit_output: {post_event_orbit_output}\n")

In [ ]:
# If orbit download doesn't work, do this ...
#pre_event_orbit_output=pre_event_split
#post_event_orbit_output=post_event_split

<font face="Calibri" size="3">The new .data directories and .dim files have had <b>"_OB"</b> appended to their filenames. Run the code cell below to <b>view the files in your current working directory for varification</b>:</font> 

In [ ]:
#optional. Uncomment to run.
#!ls --color

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.3 Project DEM into SAR Imaging Geometry </b> </font>

<font face="Calibri" size="3"> The DEM is needed for a number of processing routines in the Sentinel-1 InSAR processing workflow. Steps where the DEM is involved include:

- The accurate co-registration of master and slave image
- The removal of the topographic phase component $\phi_{topo}$
- The geocoding and terrain correction of the final InSAR data

To facilitate all of these steps, <b>Write a function to retrieve a DEM from a public repository and project the DEM into the SAR observation geometry:</b>
</font> 

In [ ]:
def back_geocoding(in_data_1, in_data_2, td2, dem=''):
    call_flag = "Back-Geocoding " 
    if dem != "":
        call_flag = f"{call_flag}-PdemName=\"External DEM\" -PexternalDEMFile={dem} "
    else: 
        call_flag = f"{call_flag}-PdemName=\"SRTM 1Sec HGT\" " 
    out = f"-t {td2}/{in_data_1.replace('.dim', '_BG')} "
    in_d = f"-SsourceProducts=\"{td2}/{in_data_1}\" \"{td2}/{in_data_2}\" "
    cmd = f"{base_snap}{call_flag}{out}{in_d}"
    print("Back-Geocoding.\nThis may take a few minutes...") 
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print(f"\nDone.")
    return f"{in_data_1.replace('.dim', '_BG.dim')}"

<font face="Calibri" size="3"> Use our back_geocoding() function to <b>retrieve and project a DEM into SAR observation geometry:</b></font> 

In [ ]:
back_geocoded_output = back_geocoding(pre_event_orbit_output, post_event_orbit_output, os.getcwd(), '')
print(f"\nback_geocoded_output: {back_geocoded_output}\n")

<font face="Calibri" size="3">The new .dim file and .data directory have had <b>"_BG"</b> appended to their names. Run the code cell below to <b>view the contents of your current working directory for varification</b>:</font> 

In [ ]:
#optional. Uncomment to run.
#!ls --color

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.4 Accurate Co-Registration using Enhanced Spectral Diversity (ESD) </b> </font>

<font face="Calibri" size="3"> For interferometric processing, two or more images must be co-registered into a common geometry. The geometry of one image is selected as the <i>master</i> geometry while the other images are often referred to as <i>slave</i> images. The pixels in slave images will be moved to align with the master image to sub-pixel accuracy.

Coregistration ensures that each ground target contributes to the same (range, azimuth) pixel in both the master and the slave image. For TOPS InSAR, co-registration using the Enhanced Spectral Diversity (ESD) method (published in <cite><a href="https://ieeexplore.ieee.org/abstract/document/6130599"><i>TOPS Interferometry With TerraSAR-X</i></a> by P. Prats-Iraola et al.</cite>) should be used. This approach can achieve co-registration qualities down to $\frac{1}{1000}$ of a pixel. 
</font> 
<br><br>
<font face="Calibri" size="3"><b>Write a function to co-register the images using ESD:</b></font> 

In [ ]:
def apply_ESD(in_data, td2):
    call_flag = "Enhanced-Spectral-Diversity " 
    back_geocoded_output = f"{td2}/{in_data.replace('.dim', '_ESD')} "
    in_d = f"-Ssource={td2}/{in_data}"
    cmd = f"{base_snap}{call_flag}-t {back_geocoded_output}{in_d}"
    print(f"Performing ESD on {in_data}")  
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print(f"\nDone.")
    test = f"{back_geocoded_output.strip()}.dim"
    if not os.path.isfile(test):
        print("\n*** applyESD Failed ***") 
        print(f"Error: Expected output file not found: {test}")
    return f"{in_data.replace('.dim','_ESD.dim')}"

<font face="Calibri" size="3">Use the apply_ESD() function to <b>co-register the images using ESD:</b></font>

In [ ]:
esd_output = apply_ESD(back_geocoded_output, os.getcwd())
print(f"\nesd_output: {esd_output}\n")

<font face="Calibri" size="3">The new .dim file and .data directory have had <b>"_ESD"</b> appended to their names. Run the code cell below to <b>view the contents of your current working directory for varification</b>:</font> 

In [ ]:
#optional. Uncomment to run.
#!ls --color

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.5 Interferogram Formation and Coherence Estimation </b> </font>

<font face="Calibri" size="3"> The interferogram is formed by cross-multiplying the master image with the complex conjugate of the slave. The amplitude of both images is multiplied while their respective phases are differenced to form the interferogram. 
The phase difference map, i.e., interferometric phase at each SAR image pixel depends only on the difference in the travel paths from each of the two SARs to the considered resolution cell.
    
Removal of the flat-Earth phase is done automatically during the interferogram formation step. The flat-Earth phase is the phase present in the interferometric signal due to the curvature of the reference surface. The flat-Earth phase is estimated using orbital and metadata information and subtracted from the complex interferogram.
</font> 
<br><br>
<font face="Calibri" size="3"><b>Write a function to create the interferogram:</b></font>

In [ ]:
def create_interferogram(in_data, td2):
    call_flag = "Interferogram " 
    out = f"{td2}/{in_data.replace('.dim', '_INT')} "
    in_d = f"-SsourceProduct={td2}/{in_data}"
    cmd = f"{base_snap}{call_flag}-t {out}{in_d}"
    print(f"Creating Interferogram from {in_data}") 
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print(f"\nDone.")
    test = f"{out.strip()}.dim"
    if not os.path.isfile(test):
        print("\n*** createInterferogram Failed ***") 
        print(f"Error: Expected output file not found: {test}")
    return f"{in_data.replace('.dim', '_INT.dim')}"

<font face="Calibri" size="3">Use the create_interferogram() to <b>co-register the images using ESD:</b></font>

In [ ]:
interferogram_output = create_interferogram(esd_output, os.getcwd())
print(f"\ninterferogram_output: {interferogram_output}\n")

<font face="Calibri" size="3">The new .dim file and .data directory have had <b>"_INT"</b> appended to their names. Run the code cell below to <b>view the contents of your current working directory for varification</b>:</font> 

In [ ]:
#optional. Uncomment to run.
#!ls --color

<br>
<hr>
<img src="NotebookAddons/Lab5-bursts.jpg" width="400" align="right" /><font face="Calibri" size="4"> <b> 3.6 TOPS Deburst </b> </font>

<font face="Calibri" size="3"> Within a subswath, TOPS data are acquired in bursts. Each burst is separated by demarcation zones. The image to your right demonstrates what the burst demarcations look like in the SAR amplitude image. Any ‘data’ within the demarcation zones can be considered invalid and should be zero-filled but may contain garbage values. 
<br><br>
<br><br>
<b>Write a function that uses the TOPS Deburst operator to remove the demarcation zones between bursts</b> in order to seamlessly join all bursts in a swath into a single image:</font> 

In [ ]:
def topsar_deburst(in_data, td2):
    call_flag = "TOPSAR-Deburst " 
    out = f"{td2}/{in_data.replace('.dim', '_DB')} "
    in_d = f"-Ssource={td2}/{in_data}"
    cmd = f"{base_snap}{call_flag}-t {out}{in_d}"
    print(f"Applying TOPSAR Deburst to {in_data}")  
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print(f"\nDone.")
    test = f"{out.strip()}.dim"
    if not os.path.isfile(test):
        print("\n*** topsarDeburst Failed ***") 
        print(f"Error: Expected output file not found: {test}")
    return f"{in_data.replace('.dim', '_DB.dim')}"

<font face="Calibri" size="3">Use the topsar_deburst() function to <b>join the bursts into a seamless image:</br></font> 

In [ ]:
topsar_deburst_output = topsar_deburst(interferogram_output, os.getcwd())
print(f"\ntopsar_deburst_output: {topsar_deburst_output}\n")

<font face="Calibri" size="3">The new .dim file and .data directory have had <b>"_DB"</b> appended to their names. Run the code cell below to <b>view the contents of your current working directory for varification</b>:</font> 

In [ ]:
#optional. Uncomment to run.
#!ls --color

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.7 Remote Topographic Phase </b> </font>

<font face="Calibri" size="3"> To emphasize phase signatures related to deformation, topographic phase contributions are typically removed using a known DEM. In SNAP, the Topographic Phase Removal operator will simulate an interferogram based on a reference DEM and subtract it from the processed interferogram.
    
SNAP will automatically find and download the DEM segment required for correcting your interferogram of interest. After topographic phase removal, the resulting product will appear largely devoid of topographic influence. 
</font> 
<br><br>
<font face="Calibri" size="3"><b>Write a function using the Topographic Phase Removal operator to remove topographic influence from the product.</b>:</font> 

In [ ]:
def topo_phase_remove(in_data, td2, dem=''):
    call_flag = "TopoPhaseRemoval " 
    if dem != "":
        call_flag = f"{call_flag}-PdemName=\"External DEM\" -PexternalDEMFile={dem} "
    else: 
        call_flag = f"{call_flag}-PdemName=\"SRTM 1Sec HGT\" " 
    out = f"{td2}/{in_data.replace('.dim', '_Topo')} " 
    in_d = f"-SsourceProduct=\"{td2}/{in_data}\" "
    cmd = f"{base_snap}{call_flag}-t {out}{in_d}"
    print(f"Removing Topographic Phase from {in_data}\nThis may take a couple minutes...") 
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print(f"\nDone.")
    test = f"{out.strip()}.dim"
    if not os.path.isfile(test):
        print("\n*** topoPhaseRemoval Failed ***") 
        print(f"Error: Expected output file not found: {test}")
    return f"{in_data.replace('.dim', '_Topo.dim')}"

<font face="Calibri" size="3">Use topo_phase_remove() to <b>remove topographic influence from the product.</b>:</font> 

In [ ]:
topo_phase_remove_output = topo_phase_remove(topsar_deburst_output,os.getcwd(),'')
print(f"\ntopo_phase_remove_output: {topo_phase_remove_output}\n")

<font face="Calibri" size="3">The new .dim file and .data directory have had <b>"_Topo"</b> appended to their names. Run the code cell below to <b>view the contents of your current working directory for varification</b>:</font> 

In [ ]:
#optional. Uncomment to run.
#!ls --color

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.8 Phase Filtering and Multi-looking </b> </font>

<img src="NotebookAddons/Lab5-filtered.jpg" width="400" align="right" /><font face="Calibri" size="3"> You will see in the GeoTIFF files you just created that up to this stage, your interferogram looks very noisy and fringe patterns are difficult to discern. Hence, we will apply two subsequent processing steps to reduce noise and enhance the appearance of the deformation fringes.

As discussed in Lecture 12, interferometric phase can be corrupted by noise from:

- Temporal decorrelation
- Geometric decorrelation
- Volume scattering
- Processing error

To be able to properly analyze the phase signatures in the interferogram, the signal-to-noise ratio will be increased by applying <b>phase filtering and multilooking</b>. Once these steps are concluded, your interferogram of the Kumamoto Earthquake should look like the example to the right.

<hr>
<b>In a first step</b> we use multilooking to improve phase fidelity. This process averages a user-specified number of range and azimuth pixels, resulting in a reduction of noise. We will use $12 \times 4$ range/azimuth looks, resulting in a pixel size of about 60m (Figure 9).</font> 
<br><br>
<font face="Calibri" size="3"><b>Write a function using multilooking to improve phase fidelity:</b></font> 

In [ ]:
def multi_look(in_data, td2):
    call_flag = f"Multilook -PnRgLooks=12 -PnAzLooks=4 "
    out = f"{td2}/{in_data.replace('.dim','_ML')} "
    in_d = f"-Ssource={td2}/{in_data}"
    cmd = f"{base_snap}{call_flag}-t {out}{in_d}"
    print(f"Applying Multilook to {in_data}") 
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print(f"\nDone.")
    test = f"{out.strip()}.dim"
    if not os.path.isfile(test):
        print("\n*** applyML Failed ***") 
        print(f"Error: Expected output file not found: {test}")
    return f"{in_data.replace('.dim', '_ML.dim')}"

In [ ]:
multi_look_output = multi_look(topo_phase_remove_output, os.getcwd())
print(f"\nmulti_look_output: {multi_look_output}\n")

<font face="Calibri" size="3">The new .dim file and .data directory have had <b>"_ML"</b> appended to their names. Run the code cell below to <b>view the contents of your current working directory for varification</b>:</font> 

In [ ]:
#optional. Uncomment to run.
#!ls --color

<hr>
<font face="Calibri" size="3">
<b>In a second step</b> we perform a phase filtering step using the state-of-the art (circa 2019) filtering approach. Specifically, we will be using the Goldstein Phase Filtering published in <cite><a href="https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/1998GL900033%4010.1002/%28ISSN%291944-8007.GRL40"><i>Radar Interferogram Filtering for Geophysical Applications</i></a> by Goldstein & Werner</cite>.
</font> 
<br><br>
<font face="Calibri" size="3"><b>Write a function using Goldstein Phase Filtering to further reduce noise in the GeoTIFF:</b></font> 

In [ ]:
def goldstein_filter(in_data, td2):
    call_flag = "GoldsteinPhaseFiltering -PcoherenceThreshold=0.2 " 
    out = f"{td2}/{in_data.replace('.dim','_GF')} "
    in_d = f"-SsourceProduct={td2}/{in_data}"
    cmd = f"{base_snap}{call_flag}-t {out}{in_d}"
    print(f"Goldstein Phase Filtering {in_data}") 
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print(f"\nDone.")
    test = f"{out.strip()}.dim"
    if not os.path.isfile(test):
        print("\n*** goldsteinFilter Failed ***") 
        print(f"Error: Expected output file not found: {test}")
    return f"{in_data.replace('.dim', '_GF.dim')}"

<font face="Calibri" size="3">Use the goldstein_filter() function to <b>further reduce noise in the GeoTIFF:</b></font> 

In [ ]:
goldstein_filter_output = goldstein_filter(multi_look_output, os.getcwd())
print(f"\ngoldstein_filter_output: {goldstein_filter_output}\n")

<font face="Calibri" size="3">The new .dim file and .data directory have had <b>"_GF"</b> appended to their names. Run the code cell below to <b>view the contents of your current working directory for varification</b>:</font> 

In [ ]:
#optional. Uncomment to run.
#!ls --color

<br>
<hr>
<img src="NotebookAddons/Lab5-unwrapping.jpg" width="400" align="right" /><font face="Calibri" size="4"> <b> 3.9 Phase Unwrapping </b> </font>

<font face="Calibri" size="3"> The interferometric phase $\phi$ is originally only available in its wrapped form. Phase unwrapping tries to recover the unwrapped phase $\psi$ given its observed wrapped phase $\phi = W\{\psi\}$.
    
The solution for this problem is difficult to find and needs some assumptions. The <b>most important of these assumptions is that we expect the true earth surface to be reasonably smooth</b>, meaning that strong discontinuities as shown in the right image are rare. Phase unwrapping algorithms mathematically code the statement that the left image is a more likely solution for a measured vector field than the image to the right.<br><br>

We will use the public-domain phase unwrapping code SNAPHU (<u>s</u>tatistical-cost, <u>n</u>etwork-flow <u>ph</u>ase-<u>u</u>nwrapping algorithm) first published in <cite><a href="https://ieeexplore.ieee.org/abstract/document/1036000"><i>Phase Unwrapping for Large SAR Interferograms: Statistical Segmentation and Generalized Network Models</i></a> by Chen & Zebker</cite> to achieve this task.</font> 
<br><br>
<font face="Calibri" size="3"><b>Write a function to export our .dim to SNAPHU format:</b></font> 

In [ ]:
def export_to_SNAPHU(in_data, td2):
    call_flag = "SnaphuExport " 
    out = f"-PtargetFolder={td2} "
    in_d = f"-SsourceProduct={td2}/{in_data}"
    cmd = f"{base_snap}{call_flag}{out}{in_d}"
    print(f"Exporting for Snaphu: {in_data}")
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print(f"\nDone.\n")

<font face="Calibri" size="3"><b>Write a function to call the SNAPHU :</b></font> 

In [ ]:
def call_SNAPHU(in_data, td2):
    print(f"Calling SNAPHU on {in_data}")
    snaphu_dir = f"{td2}/{in_data.replace('.dim','')}"
    snaphu_conf = f"{snaphu_dir}/snaphu.conf" 
    with open(snaphu_conf, 'r') as han:
        s_data = han.readlines()
    with open(snaphu_conf, 'w') as han:
        for i in range(0,len(s_data)):
            if 'LOGFILE' in s_data[i]:
                print("Found logfile in snaphu.conf")
                s_data[i] = s_data[i].replace('LOGFILE', '#LOGFILE')
            if i==6:
                snaphu_cmd = s_data[i].replace('#   ', '')
            han.write(s_data[i])
    cmd = f"cd snaphu_dir;snaphu_cmd"
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print(f"\nDone.\n")
    return snaphu_dir

<font face="Calibri" size="3"><b>Write a function to import from SNAPHU :</b></font> 

In [ ]:
def import_from_SNAPHU(in_data, td2, snaphu_dir):
    call_flag = "SnaphuImport "
    out = f"-t {td2}/{in_data.replace('.dim','_UNW')} "
    #snaphuDir = f".dim/{in_data.replace('.dim','')}"
    snaphu_file = glob.glob('%s/Unw*.hdr' % snaphu_dir)[0]
    #snaphuFile = glob.glob(f"{snaphuDir}/Unw*.hdr")[0]
    in_d = f"-SsourceProducts=\"{td2}/{in_data}\" \"{snaphu_file}\" "
    cmd = f"{base_snap}{call_flag}{out}{in_d}"
    print(f"Importing from Snaphu into {in_data}") 
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print(f"\nDone.\n")
    return f"{in_data.replace('.dim','_UNW.dim')}"

<font face="Calibri" size="3"><b>Write a function that calls the export, call, and import functions we just wrote to create a complete phase unwrapping function:</b></font> 

In [ ]:
def phase_unwrap(in_data, td2):
    export_to_SNAPHU(in_data, td2)
    snaphu_dir = call_SNAPHU(multi_look_output, os.getcwd())
    return import_from_SNAPHU(multi_look_output, os.getcwd(), snaphu_dir)

<font face="Calibri" size="3"><b>Call phase_unwrap() on the most processed version of our .dim (~_GF.dim):</b></font> 

In [ ]:
phase_unwrap_output = phase_unwrap(goldstein_filter_output,os.getcwd())
print(f"\nphase_unwrap_output: {phase_unwrap_output}\n")

<font face="Calibri" size="3">The new .dim file and .data directory have had <b>"_UNW"</b> appended to their names. Run the code cell below to <b>view the contents of your current working directory for varification</b>:</font> 

In [ ]:
#optional. Uncomment to run.
#!ls --color

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.10	Geocoding and Export in a User-defined Format </b> </font>

<font face="Calibri" size="3"> To make the data useful to geoscientists, the unwrapped interferometric phase image needs to be projected into a geographic coordinate system using a DEM-assisted geocoding step. 
    
In the following we first geocode the data set using the DEM. <b>We will geocode both the <i>unfiltered</i> and the <i>phase filtered</i> versions of the interferogram</b>. Subsequently, we convert the geocoded data sets into GeoTIFF format. Feel free to compare the filtered and unfiltered versions for their relative performance.

</font> 
<font face="Calibri" size="3"><b>Write a function that performs terrain correction:</b></font> 

In [ ]:
def terrain_correction(in_data, td2, dem=""):
    call_flag = "Terrain-Correction "
    out = f"{td2}/{in_data.replace('.dim','_TC')}  "
    in_d = f"-Ssource={td2}/{in_data}"
    if dem != '':
        in_d = f"{in_d} -PoutputComplex=true -PdemName=\"External DEM\" -PexternalDEMFile={dem} -PexternalDEMNoDataValue=0 -PpixelSpacingInMeter=60 "
    else:
        #inD = inD + ' -PdemName=\"SRTM 3Sec\" '
        in_d = f"{in_d} -PoutputComplex=true -PdemName=\"SRTM 1Sec HGT\" -PimgResamplingMethod=\"NEAREST_NEIGHBOUR\" "
    cmd = f"{base_snap}{call_flag}-t {out}{in_d}"
    print(f"Applying Terrain Correction to {in_data}")
    print(f"\nIssued os.system command:\n{cmd}")
    os.system(cmd)
    print(f"\nDone.")
    test = f"{out.strip()}.dim"
    if not os.path.isfile(test):
        print("\n*** terrain_correction Failed ***") 
        print(f"Error: Expected output file not found: {test}")
    return f"{in_data.replace('.dim','_TC.dim')}"

<font face="Calibri" size="3"><b>Call terrain_correction() on the phase filtered interferogram:</b></font> 

In [ ]:
filtered_terrain_correction_output = terrain_correction(goldstein_filter_output, os.getcwd(), '')
print(f"\nfiltered_terrain_correction_output: {filtered_terrain_correction_output}\n")

<font face="Calibri" size="3"><b>Call terrain_correction() on the unfiltered interferogram:</b></font> 

In [ ]:
unfiltered_terrain_correction_output = terrain_correction(multi_look_output, os.getcwd(), '')
print(f"\nunfiltered_terrain_correction_output: {unfiltered_terrain_correction_output}\n")

<font face="Calibri" size="3">The new .dim files and .data directories have had <b>"_TC"</b> appended to their names. Run the code cell below to <b>view the contents of your current working directory for varification</b>:</font> 

In [ ]:
!ls --color

<font face="Calibri" size="3"><b>---------------------</b></font> 

In [ ]:
cmd = f"cd {os.getcwd()}/{pre_event_base_granule}*GF_TC.data;python /home/jovyan/notebooks/ASF/GEOS_657_Labs/makeSNAPIntf.py i*.img q*.img "
print(f"\nIssued os.system command:\n{cmd}")
os.system(cmd)
print(f"\nDone.")

<font face="Calibri" size="3"><b>---------------</b></font> 

In [ ]:
cmd = f"cd {os.getcwd()}/{pre_event_base_granule}*Topo_ML_TC.data;python /home/jovyan/notebooks/ASF/GEOS_657_Labs/makeSNAPIntf.py i*.img q*.img "
print(f"\nIssued os.system command:\n{cmd}")
os.system(cmd)
print(f"\nDone.")

<font face="Calibri" size="3"><b>Convert the ------ to the GeoTiff format.</b></font> 

In [ ]:
cmd = f"cd {os.getcwd()}/{pre_event_base_granule}*GF_TC.data;gdal_translate -ot Byte -scale 0 1 0 255 coh*.img coh-{re.split('/',os.getcwd())[-1]}.tif "
print(f"\nIssued os.system command:\n{cmd}")
os.system(cmd)
print(f"\nCreated *GF_TC.data/coh-Kumamoto.tif")

<font face="Calibri" size="3"><b>Convert the ------ to the GeoTiff format.</b></font> 

In [ ]:
cmd = f"cd {os.getcwd()}/{pre_event_base_granule}*Topo_ML_TC.data;gdal_translate -ot Byte -scale 0 1 0 255 coh*.img coh-unfiltered-{re.split('/', os.getcwd())[-1]}.tif "
print(f"\nIssued os.system command:\n{cmd}") 
os.system(cmd)
print(f"\nCreated *Topo_ML_TC.data/coh-unfiltered-{re.split('/', os.getcwd())[-1]}.tif")

<font face="Calibri" size="3"><b>Convert the ------ to the GeoTiff format.</b></font> 

In [ ]:
cmd = f"cd {os.getcwd()}/{pre_event_base_granule}*GF_TC.data;mv phase.tif phase-{re.split('/', os.getcwd())[-1]}.tif "
print(f"\nIssued os.system command:\n{cmd}")
os.system(cmd)
print(f"\nCreated *GF_TC.data/phase-{re.split('/', os.getcwd())[-1]}.tif")

<font face="Calibri" size="3"><b>Convert the ------ to the GeoTiff format.</b></font> 

In [ ]:
cmd = f"cd {os.getcwd()}/{pre_event_base_granule}*Topo_ML_TC.data;mv phase.tif phase-unfiltered-{re.split('/',os.getcwd())[-1]}.tif "
print(f"\nIssued os.system command:\n{cmd}") 
os.system(cmd)
print(f"\nCreated *GF_TC.data/phase-unfiltered-{re.split('/',os.getcwd())[-1]}.tif")

<font face="Calibri" size="3"><b>Convert the ------ to the GeoTiff format.</b></font> 

In [ ]:
cmd = f"cd {os.getcwd()}/{pre_event_base_granule}*GF_TC.data;mv amplitude.tif amp-{re.split('/',os.getcwd())[-1]}.tif "
print(f"\nIssued os.system command:\n{cmd}") 
os.system(cmd)
print(f"\nCreated *GF_TC.data/amp-{re.split('/',os.getcwd())[-1]}.tif")

<font face="Calibri" size="3"><b>Convert the ------ to the GeoTiff format.</b></font> 

In [ ]:
cmd = f"cd {os.getcwd()}/{pre_event_base_granule}*Topo_ML_TC.data;mv amplitude.tif amplitude-unfiltered-{re.split('/',os.getcwd())[-1]}.tif "
print(f"\nIssued os.system command:\n{cmd}") 
os.system(cmd)
print(f"\nCreated *GF_TC.data/amplitude-unfiltered-{re.split('/',os.getcwd())[-1]}.tif")

In [ ]:
!mv */*.tif .
#!rm amp-*.tif #not sure why this was here ******************* ASK Dr. Meyer ****************

<font face="Calibri" size="3"> At the very end, let's clean up some of the temporary hard drive space before we leave:
</font> 

In [ ]:
!rm *.zip 
!rm -r *TS.d*
!rm -r *OB.d*
!rm -r *BG.d*
!rm -r *ESD.d*
!rm -r *INT.d*
!rm -r *DB.d*
!rm -r *Topo.d*
!rm -r *ML.d*
!rm -r *GF.d*

<img src="NotebookAddons/Lab5-geocoded.jpg" width="800" align="none" />

<br>
<hr>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #1</u>:  </font> Kumamoto Interferogram - Interpret Orientation of Motion & Rupture Zone </b> <font color='rgba(200,0,0,0.2)'> -- [6 Points] </font> </font>

<font face="Calibri" size="3"> Please download the file <i>phase-Kumamoto.tif</i> from the lab environment and visualize it either in QGIS or Google Earth. The interferometric phase shown in this file carries a wealth of information about surface deformation (strength and direction of motion) and the location of the surface rupture. The phase map is also a proxy for other earthquake-related parameters such as the energy released during an event.<br><br>

<u>Based on the phase patterns in the interferogram and considering the discussions we had in the lab, please discuss the following</u>:
<br><br>
<ol>
  <li>Add a line to your figure representing a likely location of the rupture zone of this earthquake. Discuss and justify your choice. <font color='rgba(200,0,0,0.2)'> -- [3 Points] </font></li>
    <br>
  <li>The dominant mechanism present at this earthquake is right-lateral strike slip with a component of normal faulting, resulting in a combination of horizontal slip along the fault and vertical uplift. Please discuss if you can separate these superimposed deformation signals (vertical and horizontal) using this interferogram alone? Justify your answer. If your answer is no, discuss a solution that may allow you to characterize the amount of vertical and horizontal motion. <font color='rgba(200,0,0,0.2)'> -- [3 Points] </font></li>
</ol>
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 1.1.A [2 Points]:</font> Insert figure of interferogram with rupture zone overlain </i> 

ADD FIGURE HERE!
</font>
</div>
<hr>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 1.1.B [1 Points]:</font> Discuss and justify your choice for rupture zone location </i> 

PROVIDE YOUR ANSWER HERE!
</font>
</div>
<hr>

<div class="alert alert-success"><font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 1.2 [3 Points]: </font> Can separate these superimposed deformation signals (vertical and horizontal) using this interferogram alone? </i> 

PROVIDE YOUR ANSWER HERE!
</font>
</div>
<hr>

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #2</u>:  </font> Kumamoto Interferogram - Compare InSAR Data to ShakeMap Information</b> <font color='rgba(200,0,0,0.2)'> -- [4 Points] </font> </font>

<font face="Calibri" size="3"> The Earthquake Hazards Program of the U.S. Geological Survey is providing a wealth of information about all significant earthquakes around the globe. The ShakeMap® is one of many sets of information included in this USGS feed. It was developed by the USGS to facilitate communication of earthquake information beyond just magnitude and location. By rapidly mapping out earthquake ground motions, ShakeMap portrays the distribution and severity of shaking. <br><br>

To access the ShakeMap for the Kumamoto event please visit <a href="https://earthquake.usgs.gov/earthquakes/eventpage/us20005iis/executive#executive" target="_blank">earthquake.usgs.gov/earthquakes/eventpage/us20005iis/executive#executive</a> and download the Event
KML (see below where to find the KML download on the page).
<img src="NotebookAddons/Lab5-WhereToFindShakemapkml.jpg" width="800" align="none" />
<br><br>

<u>For this assignment, please complete the following two tasks</u>:
<br>
<ol>
    <li>Overlay the ShakeMap onto your interferogram <font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li><br>
    <li>Provide a discussion on how well the map conforms with the interferogram. Where do the two sets of information match up? Are there places where they don’t match up? <font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li>
</ol>
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 2.1 [2 Points]:</font> Insert figure showing overlay of ShakeMap on the Kumamoto Interferogram </i> 

ADD FIGURE HERE!
</font>
</div>
<hr>

<div class="alert alert-success"><font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 2.2 [2 Points]: </font> Discussion on how well the map conforms with the interferogram. </i> 

PROVIDE YOUR ANSWER HERE!
</font>
</div>
<hr>

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #3</u>:  </font> Rerun Complete Processing Flow to Create Interferogram for the 2018 M7.2 Oaxaca, Mexico Earthquake </b> <font color='rgba(200,0,0,0.2)'> -- [10 Points] </font> </font>

<font face="Calibri" size="3"> The 2018 Oaxaca earthquake occurred on February 16, 2018 at 17:39 local time (23:39 UTC) in the Sierra Madre del Sur in Oaxaca state in Southern Mexico. It had a magnitude of 7.2 on the moment magnitude scale and a maximum felt intensity of VII (very strong) on the Mercalli intensity scale. The hypocenter was located at a depth of 24.6 km and approximately 37 km northeast of Pinotepa de Don Luis. There were only two reports of injuries from the earthquake, but later a military helicopter surveying the damage crashed and killed 14 people. There were also 15 people injured.<br><br>

The depth and focal mechanism solutions of the event are consistent with its occurrence on the subduction zone interface between these plates, approximately 90 km northeast of the Middle America Trench, where the Cocos plate begins its descent into the mantle beneath Mexico. In the region of this earthquake, the Cocos plate moves approximately northeastward at a rate of 60 mm/yr.
    
To access the ShakeMap for the Oaxaca event please visit <a href="https://earthquake.usgs.gov/earthquakes/eventpage/us20005iis/executive#executive" target="_blank">earthquake.usgs.gov/earthquakes/eventpage/us20005iis/executive#executive</a> and download the Event
KML.<br>

<u>For this assignment, please complete the following two tasks</u>:
<br>
<ol>
    <li>Rerun the processing flow from start to finish using the following two image frames as your pre-event and post-event data sets: <font color='rgba(200,0,0,0.2)'> -- [6 Points] </font>
    <ul>
        <li> <b>Pre-event image:</b> S1A_IW_SLC__1SDV_20180127T122625_20180127T122652_020340_022BE7_3F5E</li>
        <li> <b>Post-event image:</b> S1A_IW_SLC__1SDV_20180220T122625_20180220T122651_020690_023710_6CEF</li>
    </ul>
    </li><br>
    <li>Access the ShakeMap for the 2018 M7.2 Oaxaca, Mexico Earthquake via <a href="https://earthquake.usgs.gov/earthquakes/eventpage/us2000d3km/executive#executive" target="_blank">earthquake.usgs.gov/earthquakes/eventpage/us2000d3km/executive#executive</a>. Dowload the Event KML and overlay it on your interferogram <font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li><br>
<li>Discuss how the ShakeMap information compares to the InSAR data for this event. <font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li>
</ol>
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 3.1 [6 Points]:</font> Rerun the InSAR processing flow for the 2018 Oaxaca, Mexico event. </i> 

To do so, replace the 2nd and 5th code cell in this notebook with the the code-cells shown below. Then run through the whole workflow:
</font>
</div>
<hr>

In [ ]:
# Replace 4th code cell with this one
path = "/home/jovyan/notebooks/ASF/GEOS_657_Labs"
if path_exists(path):
    os.chdir(path)
    !mkdir -p lab_5_data/Oaxaca
    path = f"{path}/lab_5_data/Oaxaca"
    if path_exists(path):
        os.chdir(path)
print(f"current directory: {os.getcwd()}")

In [ ]:
# Replace 5th code cell with this one
pre_event = download_ASF_granule("S1A_IW_SLC__1SDV_20180127T122625_20180127T122652_020340_022BE7_3F5E", "SLC")
post_event = download_ASF_granule("S1A_IW_SLC__1SDV_20180220T122625_20180220T122651_020690_023710_6CEF", "SLC")

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 3.2 [2 Points]:</font> Download the ShakeMap and create a figure with the ShakeMap overlain on your interferogram
    
ADD FIGURE HERE!
</font>
</div>
<hr>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 3.3 [2 Points]:</font> Discuss how the ShakeMap information compares to the InSAR data for this event.
    
PROVIDE YOUR DISCUSSION HERE!
</font>
</div>
<hr>

<font face="Calibri" size="2"> <i>GEOS 657 Microwave Remote Sensing - Version 1.0 - Feb 2019 </i>
</font>